In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from skopt import gp_minimize
from skopt.space import Real
from sklearn.preprocessing import MinMaxScaler

output_dir = "Excel_RBF_Results"
os.makedirs(output_dir, exist_ok=True)

FEATURE_COLUMNS = [
    "C", "H", "O", "N", "F", "S", "System_Size", "a", "b", "c",
    "density", "PLD", "LCD", "N2_SA",
    "Probe_Accessible", "Probe_Occupiable", "Rosenbluth_Weight"
]

data = pd.read_csv("PIM_ExpFeatures.csv")
features = data[FEATURE_COLUMNS].values

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

labels_data = pd.read_csv("PIM_Qst_Labels.csv")
labels = labels_data["Qst_CO2_298K"].values  # Extract Qst labels

# Define hyperparameter search space for Bayesian Optimization (RBF Kernel)
space = [
    Real(1e-6, 1e2, "log-uniform", name="alpha"),  # Tighter range
    Real(1e-6, 1e2, "log-uniform", name="gamma")  # Kernel width
]

# Objective function for Bayesian Optimization
def objective(params):
    alpha, gamma = params
    testing_r2_scores = []

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, test_idx in kf.split(features):
        X_train, X_test = features[train_idx], features[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        model = KernelRidge(kernel="rbf", alpha=alpha, gamma=gamma)
        model.fit(X_train, y_train)
        test_predictions = model.predict(X_test)
        testing_r2_scores.append(r2_score(y_test, test_predictions))

    return -np.mean(testing_r2_scores)  # Negative because skopt minimizes

# Run Bayesian Optimization
result = gp_minimize(
    objective,
    space,
    n_calls=500,
    n_initial_points=50,
    random_state=42,
    verbose=True  # Display progress
)

# Extract the best hyperparameters
best_alpha, best_gamma = result.x
best_params = pd.DataFrame({"Alpha": [best_alpha], "Gamma": [best_gamma]})
#best_params.to_csv(os.path.join(output_dir, "Excel_RBF_best_hyperparameters.csv"), index=False)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
training_r2_scores = []
testing_r2_scores = []
all_train_actuals, all_train_preds = [], []
all_test_actuals, all_test_preds = [], []

for train_idx, test_idx in kf.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    model = KernelRidge(kernel="rbf", alpha=best_alpha, gamma=best_gamma)
    model.fit(X_train, y_train)

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    all_train_actuals.extend(y_train)
    all_train_preds.extend(train_predictions)
    all_test_actuals.extend(y_test)
    all_test_preds.extend(test_predictions)

    training_r2_scores.append(r2_score(y_train, train_predictions))
    testing_r2_scores.append(r2_score(y_test, test_predictions))

#cv_results = pd.DataFrame({
#    "Fold": range(1, len(training_r2_scores) + 1),
#    "Training R^2": training_r2_scores,
#    "Testing R^2": testing_r2_scores
#})
#cv_results.to_csv(os.path.join(output_dir, "Excel_RBF_cv_results.csv"), index=False)

average_training_r2 = np.mean(training_r2_scores)
average_testing_r2 = np.mean(testing_r2_scores)

#for dataset, actuals, preds, avg_r2, filename in zip(
#    ["Training", "Testing"],
#    [all_train_actuals, all_test_actuals],
#    [all_train_preds, all_test_preds],
#    [average_training_r2, average_testing_r2],
#    ["Excel_RBF_training_parity_plot.png", "Excel_RBF_testing_parity_plot.png"]
#):
#    plt.figure()
#    plt.scatter(actuals, preds, alpha=0.7, label=f"{dataset} Data")
#    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'k--', label="Perfect Prediction")
#    plt.xlabel("Actual Values")
#    plt.ylabel("Predicted Values")
#    plt.title(f"{dataset} Parity Plot ($R^2 = {avg_r2:.3f}$)")
#    plt.legend()
#    plt.grid()
#    plt.savefig(os.path.join(output_dir, filename))
#    plt.close()

#print(f"Results saved in {output_dir}")


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0040
Function value obtained: 0.3150
Current minimum: 0.3150
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: 0.2944
Current minimum: 0.2944
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: -0.0678
Current minimum: -0.0678
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: -0.6906
Current minimum: -0.6906
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: -0.9272
Current minimum: -0.9272
Iteration No: 6 started. Eval

In [2]:
# Final parity plot with both training and testing results
plt.figure(figsize=(6, 6))
plt.rcParams.update({'font.family': 'Times New Roman'})

# Scatter points
plt.scatter(all_train_actuals, all_train_preds, color='blue', alpha=0.7, edgecolor='k', label=f"Training ($R^2$ = {np.mean(training_r2_scores):.3f})")
plt.scatter(all_test_actuals, all_test_preds, color='red', alpha=0.7, edgecolor='k', label=f"Testing ($R^2$ = {np.mean(testing_r2_scores):.3f})")

# Diagonal reference line
min_val = min(min(all_train_actuals), min(all_test_actuals))
max_val = max(max(all_train_actuals), max(all_test_actuals))
plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)

# Labels and title
plt.xlabel("Actual Qst", fontsize=14)
plt.ylabel("Predicted Qst", fontsize=14)
plt.title("Radial Basis Function (rbf) KRR Prediction Results", fontsize=14)

# Styling
plt.xticks(fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend(loc='upper left', fontsize=12, frameon=False)
plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()

# Save as PDF
plt.savefig(os.path.join(output_dir, "rbf_combined_parity_plot.pdf"), format='pdf')
plt.close()
